In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
from importlib import reload
from mppi import InputFiles as I, Calculators as C, Datasets as D

Test some features of the actual implementation of YamboIn.

WE choose a folder that contain the SAVE folder.

TODO : YamboIn can manage the generation of the SAVE folder from a nscf one, the building of the yambo_folder
       and the copy of the SAVE in the yambo_folder

In [15]:
yambo_folder = 'yambo_hf'

In [54]:
#yambo = I.YamboIn('yambo -x -V qp',folder=yambo_folder)
yambo = I.YamboIn('yambo -d -k hartree -g n -p p',folder=yambo_folder)

In [55]:
print(yambo)

em1d
gw0
HF_and_locXC
ppa
X_Threads = 0.000000e+00 
DIP_Threads = 0.000000e+00 
SE_Threads = 0.000000e+00 
EXXRLvcs = 51.000000 RL
NGsBlkXp = 1.000000 RL
PPAPntXp = 27.211380 eV
GDamping = 0.100000 eV
dScStep = 0.100000 eV
Chimod =  'HARTREE'
DysSolver =        'n'
% BndsRnXp
 1 | 10 |   
%
% LongDrXp
 1.0 | 0.0 | 0.0 |   
%
% GbndRnge
 1 | 10 |   
%
% QPkrange
 1 | 32 | 1 | 10 |   
%



The variables of the input are stored in a dictionary

In [56]:
yambo.variables

{'X_Threads': [0.0, ''],
 'DIP_Threads': [0.0, ''],
 'SE_Threads': [0.0, ''],
 'EXXRLvcs': [51.0, 'RL'],
 'NGsBlkXp': [1.0, 'RL'],
 'PPAPntXp': [27.21138, 'eV'],
 'GDamping': [0.1, 'eV'],
 'dScStep': [0.1, 'eV'],
 'Chimod': 'HARTREE',
 'DysSolver': 'n',
 'BndsRnXp': [[1, 10], ''],
 'LongDrXp': [[1.0, 0.0, 0.0], ''],
 'GbndRnge': [[1, 10], ''],
 'QPkrange': [[1, 32, 1, 10], '']}

and can be modified from the python

In [57]:
yambo['QPkrange'] = [[1,32,1,6],'']

In [58]:
yambo.variables

{'X_Threads': [0.0, ''],
 'DIP_Threads': [0.0, ''],
 'SE_Threads': [0.0, ''],
 'EXXRLvcs': [51.0, 'RL'],
 'NGsBlkXp': [1.0, 'RL'],
 'PPAPntXp': [27.21138, 'eV'],
 'GDamping': [0.1, 'eV'],
 'dScStep': [0.1, 'eV'],
 'Chimod': 'HARTREE',
 'DysSolver': 'n',
 'BndsRnXp': [[1, 10], ''],
 'LongDrXp': [[1.0, 0.0, 0.0], ''],
 'GbndRnge': [[1, 10], ''],
 'QPkrange': [[1, 32, 1, 6], '']}

The runlevel specification are written as

In [59]:
yambo.arguments

['em1d', 'gw0', 'HF_and_locXC', 'ppa']

The arguments passed to yambo are

In [60]:
yambo.yamboargs

'yambo -d -k hartree -g n -p p'

The file modified by the python can be written on file as

In [61]:
yambo.write(os.path.join(yambo_folder,'lalla.in'))

The file can be processed again by yambo to reintroduce to original format

In [62]:
action = 'cd %s; %s -F %s'%(yambo_folder,yambo.yamboargs,'lalla.in')

In [63]:
print(action)

cd yambo_hf; yambo -d -k hartree -g n -p p -F lalla.in


In [64]:
os.system(action)

0

This operation will be added in the write method of the YamboIn class

The yambo object can be instanciated directly from a file

In [96]:
yambo2 = I.YamboIn(filename='yambo_hf/lalla.in')

In [97]:
print(yambo2)

em1d
gw0
HF_and_locXC
ppa
X_Threads = 0.000000e+00 
DIP_Threads = 0.000000e+00 
SE_Threads = 0.000000e+00 
EXXRLvcs = 51.000000 RL
NGsBlkXp = 1.000000 RL
PPAPntXp = 27.211380 eV
GDamping = 0.100000 eV
dScStep = 0.100000 eV
Chimod =  'HARTREE'
DysSolver =        'n'
% BndsRnXp
 1 | 10 |   
%
% LongDrXp
 1.0 | 0.0 | 0.0 |   
%
% GbndRnge
 1 | 10 |   
%
% QPkrange
 1 | 32 | 1 | 6 |   
%



In [100]:
#yambo2.variables['QPkrange'] = [[1,32,1,7],'']
yambo2['QPkrange'] = [[1,32,1,8],'']

In [101]:
print(yambo2)

em1d
gw0
HF_and_locXC
ppa
X_Threads = 0.000000e+00 
DIP_Threads = 0.000000e+00 
SE_Threads = 0.000000e+00 
EXXRLvcs = 51.000000 RL
NGsBlkXp = 1.000000 RL
PPAPntXp = 27.211380 eV
GDamping = 0.100000 eV
dScStep = 0.100000 eV
Chimod =  'HARTREE'
DysSolver =        'n'
% BndsRnXp
 1 | 10 |   
%
% LongDrXp
 1.0 | 0.0 | 0.0 |   
%
% GbndRnge
 1 | 10 |   
%
% QPkrange
 1 | 32 | 1 | 8 |   
%



Perform some tests to understand how the YamboIn class works

In [142]:
########################################

In [9]:
from subprocess import Popen, PIPE
import os, json, re
from sys import exit

class YamboIn():
    
    #Regular expressions
    _variaexp   = '([A-Za-z\_0-9]+(?:\_[A-Za-z]+)?)' #variables names
    _numexp     = '([+-]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?)' #number
    _spacexp    = '(?:[ \t]+)?' #space
    _stringexp  = '["\']([a-zA-Z0-9_ ]+?)["\']' #string
    _arrayexp   = '%'+_spacexp+_variaexp+'\s+(?:\#.+)?((?:(?:\s|\.|[+-]?\d)+?\|)+)\s+([a-zA-Z]+)?' #arrays
    _complexexp = '\('+_spacexp+_numexp+_spacexp+','+_spacexp+_numexp+_spacexp+'\)' #complex numbers
    _runexp     = '([a-zA-Z0-9_]+)' #runlevels
    # list of available runlevels to be stored in the arguments array
    _runlevels  = ['rim_cut','chi','em1s','bse','optics','bsk','bss','em1d','gw0','HF_and_locXC','setup',
                   'ppa','cohsex','life','collisions','negf','el_ph_scatt','el_el_scatt','excitons',
                   'wavefunction','fixsyms','QPDBs', 'QPDB_merge','RealTime','RT_X','RToccDos',
                   'RToccBnd','RToccEner','RToccTime','RTlifeBnd','amplitude','bzgrids','Random_Grid',
                   'gkkp','el_ph_corr','WRbsWF','Select_energy', 'RTDBs','photolum','kpts_map',
                   'RTtime','RToccupations','RTfitbands']

    def __init__(self,args='',folder='.',filename='yambo.in'):
        """
        Initalize the class
        """
        self.folder = folder
        self.yamboargs = args

        self.variables = {} #for the variables
        self.arguments = [] #for the runlevels arguments

        # if we initalize the class with arguments we call yambo to generate the input file
        if args != '':
            workdir = os.getcwd()
            os.chdir(folder)
            os.system('rm -f %s'%filename)
            yambo = Popen(args, stdout=PIPE, stderr=PIPE, stdin=PIPE, shell=True)
            yambo.wait()
            os.chdir(workdir)
            self.read_file(filename="%s/%s"%(folder,filename))
        else:
            if filename:
                self.read_file(filename="%s/%s"%(folder,filename))

    def __getitem__(self,key):
        """ 
        Get the value of a variable in the input file
        """
        return self.variables[key]

    def __setitem__(self,key,value):
        """ 
        Set the value of a variable in the input file
        """
        #if the units are not specified, add them
        if type(value) == list and str not in map(type,value):
            value = [value,'']
        if type(value) in [int,float,complex]:
            value = [value,'']
        self.variables[key] = value

    #def __delitem__(self,key):
    #    """ 
    #    Remove a keyword from the dicitonary
    #    """
    #    del self.variables[key]

    def read_file(self,filename='yambo.in'):
        """ 
        Read the variables from a file
        """
        try:
            yambofile = open(filename,"r")
        except IOError:
            print('Could not read the file %s'%filename)
            print('Something is wrong, yambo did not create the input file. Or the file you are trying to read does not exist')
            print('command: %s'%self.yamboargs)
            print('folder:  %s/'%self.folder)
            exit()
        self.read_string(yambofile.read())
        yambofile.close()

    def add_dict(self,variables):
        """
        Add a dictionary containing variables to the current inputfile
        """
        self.variables.update(variables)

    def read_string(self,inputfile):
        """
        Read the input variables from a string
        """
        var_real     = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp +
                                  self._numexp + self._spacexp + '([A-Za-z]+)?',inputfile)
        var_string   = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp + self._stringexp, inputfile)
        var_array    = re.findall(self._arrayexp,inputfile)
        var_complex  = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp + self._complexexp + self._spacexp + '([A-Za-z]+)?', inputfile)
        var_runlevel = re.findall(self._runexp + self._spacexp, inputfile)

        def clean(a):
            """
            clean the variables according to the type of data
            """
            a = a.strip()
            if a.replace('.','',1).isdigit():
                if "." in a: return float(a)
                else:        return int(a)
            return a

        # Determination of the arguments
        for key in self._runlevels:
            if key in var_runlevel:
                self.arguments.append(key)

        #float variables
        for var in var_real:
            name, value, unit = var
            self[name] = [float(value),unit]

        #string variables
        for var in var_string:
            name, string = var
            self[name] = string

        #complex variables
        for var in var_complex:
            name, real, imag, unit = var
            self[name] = [complex(float(real),float(imag)),unit]

        #array variables
        for var in var_array:
            name, array, unit = var
            array = [clean(val) for val in array.split('|')[:-1]]
            self[name] = [array,unit]

        return {"arguments": self.arguments, "variables": self.variables}

    def write(self,filename='yambo.in'):
        """
        Write a yambo input file
        """
        f = open(filename,"w")
        f.write(str(self))
        f.close()

    def __str__(self):
        """
        Returns the input file as a string
        """
        s  = ""

        #arguments
        s += "\n".join(self.arguments)+'\n'

        for key,value in self.variables.items():
            if type(value)==bytes or type(value)==str:
                s+= "%s = %10s\n"%(key,"'%s'"%value)
                continue
            if type(value[0])==float:
                val, unit = value
                if val > 1e-6:
                    s+="%s = %lf %s\n"%(key,val,unit)
                else:
                    s+="%s = %e %s\n"%(key,val,unit)
                continue
            if type(value[0])==int:
                val, unit = value
                s+="%s = %d %s\n"%(key,val,unit)
                continue
            if type(value[0])==list:
                array, unit = value
                if type(array[0])==list:
                    s+='%% %s\n'%key
                    for l in array:
                        s+="%s \n"%(" | ".join(map(str,l))+' | ')
                    s+='%s'%unit
                    s+='%\n'
                else:
                    s+="%% %s\n %s %s \n%%\n"%(key," | ".join(map(str,array))+' | ',unit)
                continue
            if type(value[0])==str:
                array = value
                s+="%% %s\n %s \n%%\n"%(key," | ".join(map(lambda x: "'%s'"%x.replace("'","").replace("\"",""),array))+' | ')
                continue
            if type(value[0])==complex:
                value, unit = value
                s+="%s = (%lf,%lf) %s\n"%(key,value.real,value.imag,unit)
                continue
            raise ValueError( "Unknown type %s for variable: %s" %( type(value), key) )
        return s

In [6]:
yambo = YamboIn(filename='yambo_hf/lalla.in')

In [7]:
yambo.arguments

['em1d', 'gw0', 'HF_and_locXC', 'ppa']

In [8]:
yambo.variables

{'X_Threads': [0.0, ''],
 'DIP_Threads': [0.0, ''],
 'SE_Threads': [0.0, ''],
 'EXXRLvcs': [51.0, 'RL'],
 'NGsBlkXp': [1.0, 'RL'],
 'PPAPntXp': [27.21138, 'eV'],
 'GDamping': [0.1, 'eV'],
 'dScStep': [0.1, 'eV'],
 'Chimod': 'HARTREE',
 'DysSolver': 'n',
 'BndsRnXp': [[1, 10], ''],
 'LongDrXp': [[1.0, 0.0, 0.0], ''],
 'GbndRnge': [[1, 10], ''],
 'QPkrange': [[1, 32, 1, 6], '']}

In [10]:
yambo.__dict__

{'folder': '.',
 'yamboargs': '',
 'variables': {'X_Threads': [0.0, ''],
  'DIP_Threads': [0.0, ''],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [51.0, 'RL'],
  'NGsBlkXp': [1.0, 'RL'],
  'PPAPntXp': [27.21138, 'eV'],
  'GDamping': [0.1, 'eV'],
  'dScStep': [0.1, 'eV'],
  'Chimod': 'HARTREE',
  'DysSolver': 'n',
  'BndsRnXp': [[1, 10], ''],
  'LongDrXp': [[1.0, 0.0, 0.0], ''],
  'GbndRnge': [[1, 10], ''],
  'QPkrange': [[1, 32, 1, 6], '']},
 'arguments': ['em1d', 'gw0', 'HF_and_locXC', 'ppa']}

In [ ]:
##################################################################

In [225]:
from subprocess import Popen, PIPE
import os, re
from sys import exit

class myYamboIn(dict):
    
    #Regular expressions
    _variaexp   = '([A-Za-z\_0-9]+(?:\_[A-Za-z]+)?)' #variables names
    _numexp     = '([+-]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?)' #number
    _spacexp    = '(?:[ \t]+)?' #space
    _stringexp  = '["\']([a-zA-Z0-9_ ]+?)["\']' #string
    _arrayexp   = '%'+_spacexp+_variaexp+'\s+(?:\#.+)?((?:(?:\s|\.|[+-]?\d)+?\|)+)\s+([a-zA-Z]+)?' #arrays
    _complexexp = '\('+_spacexp+_numexp+_spacexp+','+_spacexp+_numexp+_spacexp+'\)' #complex numbers
    _runexp     = '([a-zA-Z0-9_]+)' #runlevels
    # list of available runlevels to be stored in the arguments array
    _runlevels  = ['rim_cut','chi','em1s','bse','optics','bsk','bss','em1d','gw0','HF_and_locXC','setup',
                   'ppa','cohsex','life','collisions','negf','el_ph_scatt','el_el_scatt','excitons',
                   'wavefunction','fixsyms','QPDBs', 'QPDB_merge','RealTime','RT_X','RToccDos',
                   'RToccBnd','RToccEner','RToccTime','RTlifeBnd','amplitude','bzgrids','Random_Grid',
                   'gkkp','el_ph_corr','WRbsWF','Select_energy', 'RTDBs','photolum','kpts_map',
                   'RTtime','RToccupations','RTfitbands']

    def __init__(self,args='',folder='.',filename='yambo.in'):
        """
        Initalize the class
        """
        dict.__init__(self,args=args,folder=folder,filename=filename)
        
        # if args is not empty call yambo to generate the filename input file
        if args != '':
            workdir = os.getcwd()
            os.chdir(folder)
            os.system('rm -f %s'%filename)
            # add -F filename so yambo generates filename with the chosen args
            args+= ' -F %s'%filename 
            yambo = Popen(args, stdout=PIPE, stderr=PIPE, stdin=PIPE, shell=True)
            yambo.wait()
            os.chdir(workdir)
            self.read_file(filename="%s/%s"%(folder,filename))
        # otherwise directly read the filename input file
        else:
            if filename:
                self.read_file(filename="%s/%s"%(folder,filename))

    def read_file(self,filename):
        """ 
        Open filename and run parseInputFile to parse the input into the dictionary
        """
        try:
            yambofile = open(filename,"r")
        except IOError:
            print('Could not read the file %s'%filename)
            print('Something is wrong, yambo did not create the input file. Or the file you are trying to read does not exist')
            print('command: %s'%self['args'])
            print('folder:  %s/'%self['folder'])
            #return
            exit()
        self.parseInputFile(yambofile.read())
        yambofile.close()

    def parseInputFile(self,inputfile):
        """
        Read the arguments and variables from the input file
        """
        arguments = []
        variables = {}
        
        var_real     = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp +
                                  self._numexp + self._spacexp + '([A-Za-z]+)?',inputfile)
        var_string   = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp + self._stringexp, inputfile)
        var_array    = re.findall(self._arrayexp,inputfile)
        var_complex  = re.findall(self._variaexp + self._spacexp + '='+ self._spacexp + self._complexexp + self._spacexp + '([A-Za-z]+)?', inputfile)
        var_runlevel = re.findall(self._runexp + self._spacexp, inputfile)

        def clean(a):
            """
            clean the variables according to the type of data
            """
            a = a.strip()
            if a.replace('.','',1).isdigit():
                if "." in a: return float(a)
                else:        return int(a)
            return a

        # Determination of the arguments
        for key in self._runlevels:
            if key in var_runlevel:
                arguments.append(key)

        #float variables
        for var in var_real:
            name, value, unit = var
            variables[name] = [float(value),unit]

        #string variables
        for var in var_string:
            name, string = var
            variables[name] = string

        #complex variables
        for var in var_complex:
            name, real, imag, unit = var
            variables[name] = [complex(float(real),float(imag)),unit]

        #array variables
        for var in var_array:
            name, array, unit = var
            array = [clean(val) for val in array.split('|')[:-1]]
            variables[name] = [array,unit]
        
        self['arguments'] = arguments
        self['variables'] = variables
        


In [226]:
yambo = myYamboIn(args='yambo -d -k hartree -g n -p p',folder='yambo_hf',filename='cuccu.in') 

yambo -d -k hartree -g n -p p -F cuccu.in


In [227]:
yambo

{'args': 'yambo -d -k hartree -g n -p p',
 'folder': 'yambo_hf',
 'filename': 'cuccu.in',
 'arguments': ['em1d', 'gw0', 'HF_and_locXC', 'ppa'],
 'variables': {'X_Threads': [0.0, ''],
  'DIP_Threads': [0.0, ''],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [51.0, 'RL'],
  'NGsBlkXp': [1.0, 'RL'],
  'PPAPntXp': [27.21138, 'eV'],
  'GDamping': [0.1, 'eV'],
  'dScStep': [0.1, 'eV'],
  'Chimod': 'HARTREE',
  'DysSolver': 'n',
  'BndsRnXp': [[1, 10], ''],
  'LongDrXp': [[1.0, 0.0, 0.0], ''],
  'GbndRnge': [[1, 10], ''],
  'QPkrange': [[1, 32, 1, 10], '']}}

In [222]:
yambo = myYamboIn(filename='yambo_hf/lalla2.in') 

Could not read the file ./yambo_hf/lalla2.in
Something is wrong, yambo did not create the input file. Or the file you are trying to read does not exist
command: 
folder:  ./


SystemExit: 

/home/marco/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [201]:
yambo

{'args': '',
 'folder': '.',
 'filename': 'yambo_hf/lalla.in',
 'arguments': ['em1d', 'gw0', 'HF_and_locXC', 'ppa'],
 'variables': {'X_Threads': [0.0, ''],
  'DIP_Threads': [0.0, ''],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [51.0, 'RL'],
  'NGsBlkXp': [1.0, 'RL'],
  'PPAPntXp': [27.21138, 'eV'],
  'GDamping': [0.1, 'eV'],
  'dScStep': [0.1, 'eV'],
  'Chimod': 'HARTREE',
  'DysSolver': 'n',
  'BndsRnXp': [[1, 10], ''],
  'LongDrXp': [[1.0, 0.0, 0.0], ''],
  'GbndRnge': [[1, 10], ''],
  'QPkrange': [[1, 32, 1, 6], '']}}

dict_values(['yambo -d -k hartree -g n -p p', 'yambo_hf', 'yambo.in', 4])

In [186]:
class pippo(dict):
    
    def __init__(self,a=1,b=2):
        dict.__init__(self,a=a,b=b)
        
        # ora guarda le istruzioni 
        self.update({'c':3}) # questa non ha effetto su dict
        #dict['c'] = 3 #da errore 'type' object does not support item assignment
        
    def upd(self,bla):
        self.update(bla)

In [187]:
bla = pippo(a=7)

In [188]:
bla

{'a': 7, 'b': 2, 'c': 3}

In [191]:
bla.upd({'cc':3})

In [192]:
bla

{'a': 7, 'b': 2, 'c': 3, 'cc': 3}